<a target="_blank" href="https://colab.research.google.com/github/alejandrogtz/cccs630-fall2023/blob/main/module12/evolutionary_algorithms_applications.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Applications of Evolutionary Algorithms

## Introduction

In this module, we will apply evolutionary algorithms to solve routing problems. Please watch the following video about the travelling salesperson problem in preparation for the module's interaction portion.

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/Ov0EetgMws4" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## Concepts

You will find a list of important concepts we will review in the module below.

- Traveling salesperson problem
- Optimization
- Routes

## Interaction

In [ ]:
import numpy as np
import random
import pandas as pd
import math
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
data = pd.read_excel('points_of_interest.xlsx', sheet_name='points_of_interest') # Load the data into a Pandas Dataframe

In [ ]:
data

In [ ]:
data['Longitude']

In [ ]:
data.Longitude

In [ ]:
data.loc[0] # Access the first row via its primary label

In [ ]:
np.random.seed(1)  # For reproducible results

In [ ]:
data = data.sample(n=50)

In [ ]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.Longitude, data.Latitude))

# Set the coordinate reference system (CRS) to WGS84 (epsg:4326)
gdf.crs = "EPSG:4326"

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
gdf.to_crs(epsg=3857).plot(ax=ax, color='blue', marker='o', markersize=50)

# Add basemap
#ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

# Show the plot
plt.show()

In [ ]:
def geographic_to_cartesian(row):
    # Convert latitude and longitude from degrees to radians
    
    lat = row['Longitude']
    lon = row['Latitude']
    lat, lon = map(math.radians, [lat, lon])

    # Earth radius in kilometers
    R = 6371

    # Convert to Cartesian coordinates
    x = R * math.cos(lat) * math.cos(lon)
    y = R * math.cos(lat) * math.sin(lon)
    z = R * math.sin(lat)

    return x, y, z

In [ ]:
data['x'], data['y'], data['z']  = zip(*data.apply(geographic_to_cartesian, axis=1))

In [ ]:
data

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(data['x'], data['y'], data['z'], c='b', marker='o')

ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')

plt.title('3D Scatter Plot')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from matplotlib import rc

rc('animation', html='html5')

# Generate some random data for demonstration
np.random.seed(42)
n_points = 100
x = np.random.rand(n_points)
y = np.random.rand(n_points)
z = np.random.rand(n_points)

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create the scatter plot
scatter = ax.scatter(data['x'], data['y'], data['z'], c='b', marker='o')

# Function to update the view when called
def update(num, scatter, ax):
    ax.view_init(elev=num, azim=num)

# Create an animation by rotating the view
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), fargs=(scatter, ax), interval=50)

# Display the animation
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from matplotlib import rc

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create the scatter plot
scatter = ax.scatter(data['x'], data['y'], data['z'], c='b', marker='o')

# Function to update the view when called
def update(num, scatter, ax):
    ax.view_init(elev=num, azim=num)

# Create an animation by rotating the view
ani = FuncAnimation(fig, update, frames=np.arange(0, 360, 2), fargs=(scatter, ax), interval=50)

# Save the animation as a GIF
ani.save("rotation_animation.gif", writer="pillow")

# Display the animation
plt.show()

In [ ]:
coordinates = data[['x', 'y','z']].to_numpy()

In [ ]:
coordinates

In [ ]:
# Greedy approach
def greedy_tsp(coordinates):
    remaining_locations = list(range(len(coordinates)))
    path = [remaining_locations.pop(0)] # Start from the first location
    while remaining_locations:
        current_location = path[-1]
        next_location = min(remaining_locations, key=lambda location: np.linalg.norm(coordinates[current_location] - coordinates[location]))
        path.append(next_location)
        remaining_locations.remove(next_location)
    return path

In [ ]:
greedy_route = greedy_tsp(coordinates)

In [ ]:
# Calculate the total distance of the travel route

# Calculate Euclidean distance between two locations
def distance(from_location, to_location):
    return np.linalg.norm(coordinates[from_location] - coordinates[to_location])

# Evaluate the total distance of the route
def evaluate(route):  
    distance_sum = sum(distance(route[i], route[i+1]) for i in range(len(route) - 1))
    distance_sum += distance(route[0], route[-1])  # Return to the starting location
    return distance_sum,

In [ ]:
print("Route (Greedy Approach): ", greedy_route)
print("Route Distance (Greedy Approach): ", evaluate(greedy_route))

In [ ]:
from deap import base, creator, tools, algorithms

num_locations = len(coordinates)

# Set up DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(num_locations), num_locations)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

def main():
    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 500, stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()
    best_route = hof[0]
    print(f"Best Route (Genetic Algorithm): {best_route}")
    print(f"Route Distance (Genetic Algorithm): {evaluate(best_route)[0]}")

## Assignment 

### Conceptual Option

Pending.

### Hands-on Option

Pending.